In [ ]:
#!/usr/bin/env python
# coding: utf-8
#@Author: Wajahat Waheed
import math
import tensorflow as tf
import numpy as np
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.framework.python.ops import audio_ops
from tensorflow.contrib import ffmpeg
from scipy.fftpack import rfft, irfft
from glob import iglob
from pydub import AudioSegment
import pickle
DATA_FILES_MP3 = 'audio'
DATA_FILES_WAV = r'F:\tfaudio\audiowav' #This will be your directory of the training audio waves
file_arr = []
curr_batch = 0
def convert_mp3_to_wav():
    index = 0
    for file in iglob(DATA_FILES_MP3 + '/*.mp3'):
        mp3_to_wav = AudioSegment.from_mp3(file)
        mp3_to_wav.export(DATA_FILES_WAV+'/'+str(index)+'.wav', format='wav')
        index += 1
def process_wav():
    file_range = 0
    for file in iglob(DATA_FILES_WAV +'\*.wav'):
        file_arr.append(file)
def get_next_batch(curr_batch, songs_per_batch, sess):
    wav_arr_ch1 = []
    wav_arr_ch2 = []
    if (curr_batch) >= (len(file_arr)):
        curr_batch = 0
    start_position = curr_batch * songs_per_batch
    end_position = start_position + songs_per_batch
    print(start_position, end_position)
    for idx in range(start_position, end_position):
        print("See this:",file_arr[idx])
        audio_binary = tf.read_file(file_arr[idx])
        wav_decoder = audio_ops.decode_wav(audio_binary,desired_channels=2)
        sample_rate, audio = sess.run([wav_decoder.sample_rate, wav_decoder.audio])
        audio = np.array(audio)
        print('Audio shape', str(audio.shape))
        if len(audio[:, 0]) != 5314197: 
            print("yes")
            continue
        wav_arr_ch1.append(rfft(audio[:,0]))
        wav_arr_ch2.append(rfft(audio[:,1]))
        print("Returning File: " + file_arr[idx])
    return wav_arr_ch1, wav_arr_ch2, sample_rate

def save_to_wav( enc_song_ch1, enc_song_ch2, audio_arr_ch1, audio_arr_ch2, sample_rate, original_song_ch1, original_song_ch2, idty, folder, sess, dump=False):
    #sample_rate = 44100
    audio_arr_ch1 = irfft(np.hstack(np.hstack(audio_arr_ch1)))
    audio_arr_ch2 = irfft(np.hstack(np.hstack(audio_arr_ch2)))

    original_song_ch1 = irfft(np.hstack(np.hstack(original_song_ch1)))
    original_song_ch2 = irfft(np.hstack(np.hstack(original_song_ch2)))
    
    enc_song_ch1 = irfft(np.hstack(np.hstack(enc_song_ch1)))
    enc_song_ch2 = irfft(np.hstack(np.hstack(enc_song_ch2)))

    original_song = np.hstack(np.array((original_song_ch1, original_song_ch2)).T)
    audio_arr = np.hstack(np.array((audio_arr_ch1, audio_arr_ch2)).T)
    enc_song = np.hstack(np.array((enc_song_ch1, enc_song_ch2)).T)
    w = np.linspace(0, sample_rate, len(audio_arr))
    w = w[0:len(audio_arr)]
    
    plt.figure(1)
    plt.plot(w, original_song)
    plt.savefig(str(folder) + '/original.png')
    plt.plot(w, audio_arr)
    plt.xlabel('sample')
    plt.ylabel('amplitude')
    plt.savefig(str(folder) + '/compressed' + str(idty) + '.png')
    #plt.clf()
    plt.show()
    cols = 2
    rows = math.floor(len(audio_arr)/2)
    audio_arr = audio_arr.reshape(rows, cols)
    original_song = original_song.reshape(rows, cols)
    enc_song = enc_song.reshape(math.floor(len(enc_song)/2), cols) 
    if dump == True:
        from scipy.io import wavfile
        print('aga')
        wavfile.write('Reconstructedsongforcontin'+str(idty)+'.wav', sample_rate, audio_arr)
        wavfile.write('origal_songforcontin.wav', sample_rate, original_song)
        pickle.dump(enc_song, open( "enc_song.p", "wb" ) )
        
def next_batch(c_batch, batch_size, sess):
    ch1_arr = []
    ch2_arr = []
    wav_arr_ch1, wav_arr_ch2, sample_rate = get_next_batch(c_batch, batch_size, sess)

    for sub_arr in wav_arr_ch1:
        batch_size_ch1 = math.floor(len(sub_arr)/inputs)
        sub_arr = sub_arr[:(batch_size_ch1*inputs)]
        ch1_arr.append(np.array(sub_arr).reshape(batch_size_ch1, inputs))

    for sub_arr in  wav_arr_ch2:
        batch_size_ch2 = math.floor(len(sub_arr)/inputs)
        sub_arr = sub_arr[:(batch_size_ch2*inputs)]
        ch2_arr.append(np.array(sub_arr).reshape(batch_size_ch2, inputs))

    return np.array(ch1_arr), np.array(ch2_arr), sample_rate

#LOSS_OUT_FILE = 'Epoch_Loss.txt'
process_wav()
inputs = 10000
#Learning rate
lr = 0.00005
# L2 regularization
l2 = 0.00005
hidden_1_size = 7000
hidden_2_size = 4440
hidden_3_size = 1800
epochs = 50000
batch_size = 1
batches = 1
X = tf.placeholder(tf.float32, shape=[None, inputs])
l2_regularizer = tf.contrib.layers.l2_regularizer(l2)
#saver = tf.train.Saver()
autoencoder_dnn = partial(tf.layers.dense, activation = tf.nn.elu,kernel_initializer = tf.contrib.layers.variance_scaling_initializer(),kernel_regularizer=  tf.contrib.layers.l2_regularizer(l2))
hidden_1 = autoencoder_dnn(X, hidden_1_size)
hidden_2 = autoencoder_dnn(hidden_1, hidden_2_size)
hidden_4 = autoencoder_dnn(hidden_2, hidden_3_size)
hidden_5 = autoencoder_dnn(hidden_4, hidden_2_size)
outputs =  autoencoder_dnn(hidden_5, inputs, activation=None)
#encoder = autoencoder_dnn(inputs, hidden_4, activation=None)
#decoder = autoencoder_dnn(hidden_4, inputs, activation=None)
reconstruction_loss = tf.reduce_mean(tf.square(outputs-X))
reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_loss)
optimizer = tf.train.AdamOptimizer(lr)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

##### Run training
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./model.ckpt")
    ch1_song, ch2_song, sample_rate = next_batch(4, batch_size, sess)
    for epoch in range(epochs):
        epoch_loss = []
        print("Epoch: " + str(epoch))
        for i in range(batches):      
          # print("ch1_song",ch1_song) 
            total_songs = np.hstack([ch1_song, ch2_song])
            print('totalsongs' + str(total_songs.shape))
            batch_loss = []
            for j in range(len(total_songs)):
                x_batch = total_songs[j]
                _, l = sess.run([training_op, loss], feed_dict={X:x_batch})
                batch_loss.append(l)
                print("Song loss: " + str(l))

            print("Curr Epoch: " + str(epoch) + " Curr Batch: " + str(i) + "/"+ str(batches))
            print("Batch Loss: " + str(np.mean(batch_loss)))
            epoch_loss.append(np.mean(batch_loss))
        print("Epoch Avg Loss: " + str(np.mean(epoch_loss)))
        if epoch == 50:
            ch1_song_new, ch2_song_new, sample_rate_new = ch1_song, ch2_song, sample_rate
            #ch1_song_new, ch2_song_new, sample_rate_new = next_batch(0, 1,sess)
            
          # print("ch1",ch1_song_new)
          # print("ch2",ch2_song_new)
            x_batch = np.hstack([ch1_song_new, ch2_song_new])[0]
            print("Sample rate: " + str(sample_rate_new))

            orig_song = []
            full_song = []
            encoded_song = []
            encoded = hidden_4.eval(feed_dict={X: x_batch})
            print('encodedshape' + str(encoded.shape))
            evaluation = outputs.eval(feed_dict={X: x_batch})
            print("Output: " + str(evaluation))
            print('evaluation'+str(evaluation.shape))
            print('x_batch'+str(x_batch.shape))
            encoded_song.append(encoded)
            full_song.append(evaluation)
            orig_song.append(x_batch)

            # Merge the nested arrays
            orig_song = np.hstack(orig_song)
            full_song = np.hstack(full_song)
            encoded_song = np.hstack(encoded_song)

            # Compute and split the channels
            orig_song_ch1 = orig_song[:math.floor(len(orig_song)/2)]
            orig_song_ch2 = orig_song[math.floor(len(orig_song)/2):]
            full_song_ch1 = full_song[:math.floor(len(full_song)/2)]
            full_song_ch2 = full_song[math.floor(len(full_song)/2):]
            enc_song_ch1 = encoded_song[:math.floor(len(encoded_song)/2)]
            enc_song_ch2 = encoded_song[math.floor(len(encoded_song)/2):]

            # Save both the untouched song and reconstructed song to the 'output' folder
            save_to_wav(enc_song_ch1, enc_song_ch2, full_song_ch1, full_song_ch2, sample_rate, orig_song_ch1, orig_song_ch2, epoch, 'output', sess, True)
            #saver.save(sess, './model.ckpt')

W0113 00:53:51.602934 33696 deprecation.py:323] From <ipython-input-1-a9e0b4860b72>:176: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


I GOT HERE


W0113 00:54:11.267351 33696 deprecation.py:323] From c:\anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


4 5
See this: F:\tfaudio\audiowav\r (15).wav
audio shape (5314197, 2)
Returning File: F:\tfaudio\audiowav\r (15).wav
Epoch: 0
totalsongs(1, 1062, 10000)
Song loss: 2897.7048
Curr Epoch: 0 Curr Batch: 0/1
Batch Loss: 2897.7048
Epoch Avg Loss: 2897.7048
Epoch: 1
totalsongs(1, 1062, 10000)
Song loss: 2868.833
Curr Epoch: 1 Curr Batch: 0/1
Batch Loss: 2868.833
Epoch Avg Loss: 2868.833
Epoch: 2
totalsongs(1, 1062, 10000)
Song loss: 2834.8965
Curr Epoch: 2 Curr Batch: 0/1
Batch Loss: 2834.8965
Epoch Avg Loss: 2834.8965
Epoch: 3
totalsongs(1, 1062, 10000)
Song loss: 2814.4148
Curr Epoch: 3 Curr Batch: 0/1
Batch Loss: 2814.4148
Epoch Avg Loss: 2814.4148
Epoch: 4
totalsongs(1, 1062, 10000)
Song loss: 2808.1172
Curr Epoch: 4 Curr Batch: 0/1
Batch Loss: 2808.1172
Epoch Avg Loss: 2808.1172
Epoch: 5
totalsongs(1, 1062, 10000)
Song loss: 2805.7178
Curr Epoch: 5 Curr Batch: 0/1
Batch Loss: 2805.7178
Epoch Avg Loss: 2805.7178
Epoch: 6
totalsongs(1, 1062, 10000)
Song loss: 2800.1287
Curr Epoch: 6 Curr 

<Figure size 640x480 with 1 Axes>

aga
Epoch: 51
totalsongs(1, 1062, 10000)
Song loss: 2674.338
Curr Epoch: 51 Curr Batch: 0/1
Batch Loss: 2674.338
Epoch Avg Loss: 2674.338
Epoch: 52
totalsongs(1, 1062, 10000)
Song loss: 2673.1316
Curr Epoch: 52 Curr Batch: 0/1
Batch Loss: 2673.1316
Epoch Avg Loss: 2673.1316
Epoch: 53
totalsongs(1, 1062, 10000)
Song loss: 2671.9539
Curr Epoch: 53 Curr Batch: 0/1
Batch Loss: 2671.9539
Epoch Avg Loss: 2671.9539
Epoch: 54
totalsongs(1, 1062, 10000)
Song loss: 2670.7708
Curr Epoch: 54 Curr Batch: 0/1
Batch Loss: 2670.7708
Epoch Avg Loss: 2670.7708
Epoch: 55
totalsongs(1, 1062, 10000)
Song loss: 2669.581
Curr Epoch: 55 Curr Batch: 0/1
Batch Loss: 2669.581
Epoch Avg Loss: 2669.581
Epoch: 56
totalsongs(1, 1062, 10000)
Song loss: 2668.4133
Curr Epoch: 56 Curr Batch: 0/1
Batch Loss: 2668.4133
Epoch Avg Loss: 2668.4133
Epoch: 57
totalsongs(1, 1062, 10000)
Song loss: 2667.2917
Curr Epoch: 57 Curr Batch: 0/1
Batch Loss: 2667.2917
Epoch Avg Loss: 2667.2917
Epoch: 58
totalsongs(1, 1062, 10000)
Song l